In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
#import pickle
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
#from scipy.cluster.hierarchy import linkage
from sklearn.metrics import pairwise_distances
#import matplotlib.pyplot as plt
#import seaborn as sns

import settings as conf
import metadata
from utils import chunker

# Load gene associations

## Clinvar

In [3]:
clinvar_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'clinvar-gene_associations.pkl.xz')
display(clinvar_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/clinvar-gene_associations.pkl.xz'

In [4]:
clinvar_genes_associations = pd.read_pickle(clinvar_genes_associations_filename)

In [5]:
clinvar_genes_associations.shape

(4194, 5586)

In [6]:
clinvar_genes_associations.head(5)

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,"Myopia 21, autosomal dominant",Paget disease of bone 6,ZNF711-Related X-linked Mental Retardation,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
gene,,,,,,,,,,,,,,,,,,,,,
A2M,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A4GALT,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAS,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAGAB,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## S-MultiXcan

In [7]:
smultixcan_genes_associations_zscores_filename = os.path.join(conf.GENE_ASSOC_DIR, f'smultixcan-mashr-zscores.pkl.xz')
display(smultixcan_genes_associations_zscores_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan-mashr-zscores.pkl.xz'

In [8]:
smultixcan_genes_associations_zscores = pd.read_pickle(smultixcan_genes_associations_zscores_filename)

In [9]:
smultixcan_genes_associations_zscores.shape

(22515, 4091)

In [10]:
smultixcan_genes_associations_zscores.head(5)

,4270-Volume_level_set_by_participant_left,S05-Diagnoses_main_ICD10_S05_Injury_of_eye_and_orbit,20003_1141157402-Treatmentmedication_code_prednisolone_product,20002_1427-Noncancer_illness_code_selfreported_polycystic_kidney,110001-Invitation_to_complete_online_24hour_recall_dietary_questionnaire_acceptance,22617_2442-Job_SOC_coding_Social_workers,J93-Diagnoses_main_ICD10_J93_Pneumothorax,22601_41133206-Job_coding_local_government_administrative_officer_or_assistant_or_clerk,6145_3-Illness_injury_bereavement_stress_in_last_2_years_Death_of_a_close_relative,20002_1597-Noncancer_illness_code_selfreported_tinnitus_tiniitis,...,PGC_ADHD_EUR_2017,BCAC_ER_negative_BreastCancer_EUR,SSGAC_Education_Years_Pooled,pgc.scz2,MAGNETIC_HDL.C,MAGIC_ln_FastingInsulin,Astle_et_al_2016_Sum_eosinophil_basophil_counts,SSGAC_Depressive_Symptoms,Jones_et_al_2016_SleepDuration,Astle_et_al_2016_Red_blood_cell_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.059156,0.217326,0.029150,0.104664,0.553134,1.101089,0.280399,0.527449,0.625478,0.620854,...,0.024717,0.816922,0.321512,0.510834,2.336462,0.893203,0.223675,0.367414,0.178426,0.824314
ENSG00000000457,0.458818,1.037278,0.710374,2.913855,0.118886,0.194594,1.485428,0.166128,2.184210,0.855502,...,1.202984,0.472136,0.447472,2.545653,0.350052,1.251757,2.651112,0.414171,0.206143,0.237223
ENSG00000000460,0.130622,1.673556,0.662508,1.383726,0.031908,0.483261,0.797453,0.595180,1.884805,1.045412,...,1.033308,1.473851,0.458711,0.541782,0.676499,0.640565,2.843532,0.336480,0.785382,0.695624
ENSG00000000938,0.557883,1.611271,0.773417,0.571616,0.294555,1.270594,1.196635,1.613178,0.530660,0.058038,...,0.540496,1.208182,0.558368,1.886356,0.779920,0.058739,1.635829,0.056432,1.655463,1.494501
ENSG00000000971,0.054685,0.687113,0.657400,0.624363,0.589818,0.008068,0.243731,0.064759,0.286240,0.688779,...,1.598721,2.044839,2.766758,2.319072,0.773096,0.398527,1.918412,3.004544,0.971837,0.005703


### Compute unique list of genes and non-empty diseases

In [11]:
clinvar_genes_to_id = pd.Index([
    metadata.GENE_NAME_TO_ID_MAP[x]
    for x in clinvar_genes_associations.index
    if x in metadata.GENE_NAME_TO_ID_MAP
])
display(len(clinvar_genes_to_id))

shared_gene_ids = clinvar_genes_to_id.intersection(smultixcan_genes_associations_zscores.index)
display(len(shared_gene_ids))

shared_gene_names = pd.Index([metadata.GENE_ID_TO_NAME_MAP[x] for x in shared_gene_ids])
display(len(shared_gene_names))

assert len(shared_gene_ids) == len(shared_gene_names)

3790

3789

3789

In [12]:
smultixcan_genes_associations_zscores = smultixcan_genes_associations_zscores.loc[shared_gene_ids]
clinvar_genes_associations = clinvar_genes_associations.loc[shared_gene_names]

In [13]:
_tmp = clinvar_genes_associations.sum()
_clinvar_diseases_to_remove = _tmp[_tmp == 0].index
display(_clinvar_diseases_to_remove.shape)

(480,)

In [14]:
# remove diseases that have no-genes associated (after selecting just shared ones above)
clinvar_genes_associations = clinvar_genes_associations.drop(columns=_clinvar_diseases_to_remove)

In [15]:
_tmp = clinvar_genes_associations.sum()
assert _tmp[_tmp == 0].shape[0] == 0

In [16]:
display(smultixcan_genes_associations_zscores.shape)
display(clinvar_genes_associations.shape)

(3789, 4091)

(3789, 5106)

# Compute associations

In [17]:
smultixcan_squared_zscores = np.power(smultixcan_genes_associations_zscores.T.fillna(0).values, 2)

In [18]:
smultixcan_squared_zscores.shape

(4091, 3789)

In [19]:
clinvar_genes_associations.shape

(3789, 5106)

In [20]:
z2_sums = np.dot(smultixcan_squared_zscores, clinvar_genes_associations.values)

In [21]:
z2_sums.shape

(4091, 5106)

In [22]:
n_clinvar_genes = clinvar_genes_associations.sum().to_frame().T
n_clinvar_genes = pd.concat([n_clinvar_genes] * z2_sums.shape[0], ignore_index=True).values

In [23]:
n_clinvar_genes.shape

(4091, 5106)

In [24]:
pheno_corrs_results = z2_sums / n_clinvar_genes

In [25]:
pheno_corrs_results.shape

(4091, 5106)

In [26]:
assoc_coef_df = pd.DataFrame(
    pheno_corrs_results,
    index=smultixcan_genes_associations_zscores.columns.copy(),
    columns=clinvar_genes_associations.columns.copy()
)

In [27]:
assert assoc_coef_df.shape == (smultixcan_genes_associations_zscores.shape[1], clinvar_genes_associations.shape[1])
display(assoc_coef_df.shape)

(4091, 5106)

In [28]:
assoc_coef_df.head()

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,Retinitis pigmentosa 58,"Myopia 21, autosomal dominant",Paget disease of bone 6,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
4270-Volume_level_set_by_participant_left,0.512739,0.342467,1.070237,1.800286,1.631911,0.698835,0.074415,0.074415,1.387650,1.387650,...,1.017634,1.224636,0.009039,0.385532,3.521638,0.223988,0.001329,0.425832,0.011000,0.011000
S05-Diagnoses_main_ICD10_S05_Injury_of_eye_and_orbit,0.217049,0.001955,0.631155,1.087293,0.007369,2.805816,0.075216,0.075216,0.093814,0.093814,...,0.501215,1.852592,1.481549,0.046810,0.619792,0.164473,2.656553,3.644131,0.321142,0.321142
20003_1141157402-Treatmentmedication_code_prednisolone_product,0.343953,0.676443,0.034698,0.203671,0.004971,1.390301,1.228719,1.228719,2.967204,2.967204,...,0.360941,11.078803,0.831347,0.924590,0.223633,1.503475,1.331688,2.692623,0.000386,0.000386
20002_1427-Noncancer_illness_code_selfreported_polycystic_kidney,0.444715,0.231927,0.288159,3.099246,0.446979,1.205976,0.114788,0.114788,0.686694,0.686694,...,1.772919,1.062408,0.254789,1.119640,1.354223,0.025730,0.680319,1.682058,0.579198,0.579198
110001-Invitation_to_complete_online_24hour_recall_dietary_questionnaire_acceptance,1.229807,2.597971,6.239911,1.639243,1.395789,0.051032,1.202180,1.202180,3.934130,3.934130,...,2.650158,0.972351,0.661263,0.021415,0.685834,1.089133,2.498117,2.861720,0.095069,0.095069


In [29]:
# testing
_pheno0 = "22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters"
_pheno1 = "Alzheimer's disease"

_genes = clinvar_genes_associations[_pheno1].sort_values(ascending=False)
display(_genes.head(10))
_genes = _genes[_genes > 0].index
display(_genes)

A2M      1
MPO      1
APP      1
NOS3     1
PLAU     1
HFE      1
FOXH1    0
FOXF1    0
FOXG1    0
FOXL2    0
Name: Alzheimer's disease, dtype: uint8

Index(['A2M', 'MPO', 'APP', 'NOS3', 'PLAU', 'HFE'], dtype='object')

In [30]:
_tmp = smultixcan_genes_associations_zscores[_pheno0].rename(index=metadata.GENE_ID_TO_NAME_MAP)
display(_tmp.head())

A2M       0.181998
A2ML1     1.054712
A4GALT    2.209362
AAAS      0.598121
AAGAB     0.363092
Name: 22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters, dtype: float64

In [31]:
_tmp.loc[_genes]

A2M     0.181998
MPO     1.749522
APP     0.022963
NOS3    0.873820
PLAU    0.178090
HFE     0.208738
Name: 22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters, dtype: float64

In [32]:
_obs = np.power(_tmp.loc[_genes], 2).sum() / 6
display(_obs)
_exp = assoc_coef_df.loc[_pheno0, _pheno1]
display(_exp)
assert np.isclose(_obs, _exp), (_obs, _exp)

0.6555543699722598

0.6555543699722599

In [33]:
# testing
_pheno0 = '30040_raw-Mean_corpuscular_volume'
_pheno1 = "Alzheimer's disease"

_genes = clinvar_genes_associations[_pheno1].sort_values(ascending=False)
display(_genes.head(10))
_genes = _genes[_genes > 0].index
display(_genes)

A2M      1
MPO      1
APP      1
NOS3     1
PLAU     1
HFE      1
FOXH1    0
FOXF1    0
FOXG1    0
FOXL2    0
Name: Alzheimer's disease, dtype: uint8

Index(['A2M', 'MPO', 'APP', 'NOS3', 'PLAU', 'HFE'], dtype='object')

In [34]:
_tmp = smultixcan_genes_associations_zscores[_pheno0].rename(index=metadata.GENE_ID_TO_NAME_MAP)
display(_tmp.head())

A2M       1.370702
A2ML1     0.541736
A4GALT    6.618485
AAAS      6.178175
AAGAB     0.975214
Name: 30040_raw-Mean_corpuscular_volume, dtype: float64

In [35]:
_tmp.loc[_genes]

A2M      1.370702
MPO      2.919041
APP      0.544944
NOS3     2.223614
PLAU     1.466185
HFE     40.000000
Name: 30040_raw-Mean_corpuscular_volume, dtype: float64

In [36]:
_obs = np.power(_tmp.loc[_genes], 2).sum() / 6
display(_obs)
_exp = assoc_coef_df.loc[_pheno0, _pheno1]
display(_exp)
assert np.isclose(_obs, _exp), (_obs, _exp)

269.6317906903394

269.6317906903394

### Save

In [37]:
# sort index like smultixcan results
assoc_coef_df = assoc_coef_df.loc[smultixcan_genes_associations_zscores.columns]

In [38]:
assoc_coef_df.index.rename('phenomexcan_traits', inplace=True)

In [39]:
assoc_coef_df_filename = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan_and_clinvar-z2.pkl.xz')
display(assoc_coef_df_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan_and_clinvar-z2.pkl.xz'

In [40]:
assoc_coef_df.to_pickle(assoc_coef_df_filename)

In [41]:
pd.Series(assoc_coef_df.values.flatten()).describe()

count    2.088865e+07
mean     1.330285e+00
std      7.561057e+00
min      0.000000e+00
25%      1.233913e-01
50%      5.334078e-01
75%      1.485715e+00
max      1.600000e+03
dtype: float64

In [42]:
assert not assoc_coef_df.isna().any().any()

In [43]:
assert not (assoc_coef_df < 0).any().any()

In [44]:
assert np.isfinite(assoc_coef_df).all().all()

In [45]:
# some testing
ukb_pheno = '20002_1111-Noncancer_illness_code_selfreported_asthma'
clinvar_pheno = 'Asthma, susceptibility to'

assert assoc_coef_df.loc[ukb_pheno, clinvar_pheno] == 28.748716710063185

### Save for publication

In [46]:
output_file = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan_and_clinvar-z2.tsv.gz')
display(output_file)

assoc_coef_df.to_csv(output_file, sep='\t', float_format='%.4e')

'/mnt/phenomexcan_base/gene_assoc/smultixcan_and_clinvar-z2.tsv.gz'

In [47]:
_tmp = pd.read_csv(output_file, sep='\t', index_col='phenomexcan_traits')

In [48]:
display(_tmp.shape)
assert _tmp.shape == assoc_coef_df.shape

(4091, 5106)

In [49]:
_tmp.head()

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,Retinitis pigmentosa 58,"Myopia 21, autosomal dominant",Paget disease of bone 6,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
phenomexcan_traits,,,,,,,,,,,,,,,,,,,,,
4270-Volume_level_set_by_participant_left,0.51274,0.342470,1.070200,1.80030,1.631900,0.698840,0.074415,0.074415,1.387600,1.387600,...,1.01760,1.22460,0.009039,0.385530,3.52160,0.22399,0.001329,0.42583,0.011000,0.011000
S05-Diagnoses_main_ICD10_S05_Injury_of_eye_and_orbit,0.21705,0.001955,0.631150,1.08730,0.007369,2.805800,0.075216,0.075216,0.093814,0.093814,...,0.50121,1.85260,1.481500,0.046810,0.61979,0.16447,2.656600,3.64410,0.321140,0.321140
20003_1141157402-Treatmentmedication_code_prednisolone_product,0.34395,0.676440,0.034698,0.20367,0.004971,1.390300,1.228700,1.228700,2.967200,2.967200,...,0.36094,11.07900,0.831350,0.924590,0.22363,1.50350,1.331700,2.69260,0.000386,0.000386
20002_1427-Noncancer_illness_code_selfreported_polycystic_kidney,0.44471,0.231930,0.288160,3.09920,0.446980,1.206000,0.114790,0.114790,0.686690,0.686690,...,1.77290,1.06240,0.254790,1.119600,1.35420,0.02573,0.680320,1.68210,0.579200,0.579200
110001-Invitation_to_complete_online_24hour_recall_dietary_questionnaire_acceptance,1.22980,2.598000,6.239900,1.63920,1.395800,0.051032,1.202200,1.202200,3.934100,3.934100,...,2.65020,0.97235,0.661260,0.021415,0.68583,1.08910,2.498100,2.86170,0.095069,0.095069


In [50]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [51]:
assert not _tmp.isna().any().any()

In [52]:
assert ((_tmp >= 0) & (_tmp <= 1600)).all().all()

In [53]:
assert np.allclose(_tmp.values, assoc_coef_df.values, atol=1e-15, rtol=1e-4)